In [1]:
from dataset_names.dataset_names import DatasetNames
from attacks.attack_names import AttackNames
from detections.detection_names import DetectionNames
from evaluation.utils import get_output_path

In [2]:
import os
import json
from statistics import fmean

def get_metrics(base_path):
    TP_list = []
    FP_list = []
    FN_list = []
    Precision_list = []
    Recall_list = []
    malicious_clients_list = []
    detected_clients_list = []
    effectivenesses_list = []
    time_per_iteration_list = []
    for root, dirs, files in os.walk(base_path):
        relative_path = os.path.relpath(root, base_path)
        path_parts = relative_path.split(os.sep)

        # We expect 2 parts: date, time
        if len(path_parts) >= 2:
            for file_name in files:
                file_path = os.path.join(root, file_name)
                if "Precision_Recall" in file_name:
                    with open(file_path, 'r') as f:
                        precision_recall = json.load(f)
                    TP_list.append(precision_recall["TP"])
                    FP_list.append(precision_recall["FP"])
                    FN_list.append(precision_recall["FN"])
                    Precision_list.append(precision_recall["Precision"])
                    Recall_list.append(precision_recall["Recall"])
                elif "clients" in file_name:
                    with open(file_path, 'r') as f:
                        clients = json.load(f)
                    num_clients = clients["num_clients"]
                    malicious_clients = clients["malicious_clients"]
                    malicious_clients_list.append(malicious_clients)
                    detected_clients_list.append(clients["detected_clients"])
                elif "round_metrics" in file_name:
                    with open(file_path, 'r') as f:
                        round_metrics = json.load(f)
                elif "time" in file_name:
                    with open(file_path, 'r') as f:
                        time = json.load(f)
                    time_per_iteration = time["time_per_iteration"]
                    time_per_iteration_list.append(time_per_iteration)


            # Calculate the Effectiveness of the run
            acc_0 = round_metrics[0]["accuracy"]    # We use the accuracy of the first (0) round.
            acc_n = round_metrics[-1]["accuracy"]
            clients_done = []
            effectivenesses = []
            for idx, round_metric in enumerate(round_metrics):
                detected_FRs = round_metric["detected_FR"]
                acc_i_1 = round_metrics[idx-1]["accuracy"]  # Get the accuracy of the previous global model.
                if detected_FRs:
                    for det_FR in detected_FRs:
                        # Check whether the FR is malicious and if it has not already been detected in a previous round
                        if det_FR in malicious_clients and det_FR not in clients_done:
                            # In this round, a new FR was detected.
                            # Calculate the Effectiveness for this FR.
                            effectiveness = (1 - (acc_i_1/acc_n)) / (1 - (acc_0/acc_n))
                            effectivenesses.append(effectiveness)
                            clients_done.append(det_FR)
            
            not_detected_FR = [cid for cid in malicious_clients if cid not in clients_done]
            for n_d_FR in not_detected_FR:
                effectivenesses.append(0.0)
            effectiveness_all = fmean(effectivenesses)

            effectivenesses_list.append(effectiveness_all)

    TP_avg = fmean(TP_list)
    FP_avg = fmean(FP_list)
    FN_avg = fmean(FN_list)
    Precision_avg = fmean(Precision_list)
    Recalls_avg = fmean(Recall_list)
    effectiveness_avg = fmean(effectivenesses_list)
    time_per_iteration_avg = fmean(time_per_iteration_list)

    return {
        "TP_avg": TP_avg,
        "FP_avg": FP_avg,
        "FN_avg": FN_avg,
        "Precision_avg": Precision_avg,
        "Recall_avg": Recalls_avg,
        "Effectiveness_avg": effectiveness_avg,
        "time_per_iteration_avg": time_per_iteration_avg
    }

In [3]:
dataset = DatasetNames.mnist
n_clients = 100
perc_malicious = 10
attack = AttackNames.random_weights_attack
detection = DetectionNames.fgfl_detection

test_path = "./outputs/" + get_output_path(dataset, n_clients, perc_malicious, attack, detection)
metrics = get_metrics(test_path)
print(metrics)

{'TP_avg': 10.0, 'FP_avg': 3.0, 'FN_avg': 0.0, 'Precision_avg': 0.7692307692307693, 'Recall_avg': 1.0, 'Effectiveness_avg': 0.4200041315631395, 'time_per_iteration_avg': 3.05485565662384}


# MNIST experiment evaluation

In [4]:
from typing import List
def get_experiment_results(dataset: DatasetNames, n_clients: List, perc_malicious: List, attacks: List, detections: List):
    results = {}

    for n_c in n_clients:
        for perc_m in perc_malicious:
            for attack in attacks:
                for detection in detections:
                    path_name = get_output_path(dataset, n_c, perc_m, attack, detection)
                    path = "./outputs/" + path_name
                    metrics = get_metrics(path)
                    results[path_name] = metrics

## Baseline Runs

### MNIST

In [ ]:
dataset = DatasetNames.mnist
n_clients = [10, 100]
perc_malicious = [0]
attacks = [AttackNames.no_attack]
detections = [DetectionNames.no_detection]

results = get_experiment_results(dataset, n_clients, perc_malicious, attacks, detections)

### CIFAR10

In [ ]:
dataset = DatasetNames.cifar10
n_clients = [10, 100]
perc_malicious = [0]
attacks = [AttackNames.no_attack]
detections = [DetectionNames.no_detection]

results = get_experiment_results(dataset, n_clients, perc_malicious, attacks, detections)